In [1]:
#| default_exp requirementsPrompt

In [2]:
#| export
from langchain_core.messages import HumanMessage

from langchain_openai import ChatOpenAI

model = ChatOpenAI()


In [3]:
#| export 
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from hn_jobs_chat.keys import keys
captureRequirementsPrompt = """
You are a requirements recorder, helping a job searcher, by recording the requirements they have in the following areas:\n\n"""+  "\n".join(keys) + """\n\n
 Listen to the user's message, and return a json object with two keys: 'new_requirements', and 'remove_requirements'. The first one listing new requirements that the user has stated, if any, and the other listing requirements the user wants to remove from the requirements list. Use only the  list of requirements above. Do not add any new types of requirement to the list. 

For example, if the user says "I want a job in New York, you should return the following json object:

{{
  "new_requirements": {{
    "location": "New York",
  }},
  "remove_requirements": {{
  }}
}}

For example, if the user says "I want a job in California, with a salary of 100k, and I don't want a job in Ohio", you should return the following json object:

{{
  "new_requirements": {{
    "location": "New York",
    "salary": 100000
  }},
  "remove_requirements": {{
    "location": "Ohio"
  }}
}}

--------------------\n\n
"""

basePrompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            captureRequirementsPrompt,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = basePrompt | model


In [4]:
#| export
config = {"configurable": {"session_id": "abc6"}}

In [5]:
#| export 
import json
# Invoke the requirements bot with the user's message
def getRequirementsResponse(input:str):
    response = chain.invoke(input=[HumanMessage(input)])

    json_response = json.loads(response.content)

    return json_response


In [6]:

resp = getRequirementsResponse("I want a job in New York, with a salary of 100k, and I don't want a job in Ohio")

resp

{'new_requirements': {'location': 'New York', 'compensation': 100000},
 'remove_requirements': {'location': 'Ohio'}}

In [7]:
#| export

requirements = {
    'want': {},
    'donotwant': {} 
}



In [8]:
#| export
from pydantic import BaseModel

class requirementResponse(BaseModel):
    new_requirements: dict
    remove_requirements: dict


In [9]:
#| export
# TODO: dyanamically parse response into requirement keys

def handleResponse(response: requirementResponse):
    if response.new_requirements:
        for key, value in response.new_requirements.items():
            requirements["want"][key] = value

    if response.remove_requirements:
        for key, value in response.remove_requirements.items():
            requirements["donotwant"][key] = value

            if key in requirements["want"]:
                del requirements["want"][key]

    return response

In [10]:

def getRequirements():
    return requirements



In [11]:
#| export
import json
# Return a lists of desired and undesired requirements
def handleSubmit(input:str) -> str:
    json_response = getRequirementsResponse(input)
    validated_response = requirementResponse(**json_response)

    handleResponse(validated_response)

    return getRequirementsPrompt()

In [12]:
import ipywidgets as widgets
from IPython.display import display
import json

# Create input widgets
text_input = widgets.Text()

# Create a button
submit_button = widgets.Button(description='Submit')

# Define a function to handle form submission
def handle_submit(button):
   handleSubmit(text_input.value)

# Attach the function to the button's on_click event
submit_button.on_click(handle_submit)

# Display the form
display(text_input, submit_button)

Text(value='')

Button(description='Submit', style=ButtonStyle())

In [13]:
#| hide
from nbdev import nbdev_export
nbdev_export()